# StackOverflow Search Optimazation

### Importing the required libraries

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import nltk
import requests

#### Downloading the required libraries for nltk

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Cleaning the input to get tags

In [4]:
input = """how to define function in python""" 

from nltk.tokenize import word_tokenize
tokenized_word=word_tokenize(input)

from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)

#### Requesting the StackExchange API for questions using the tags obatained

In [5]:
URL = 'https://api.stackexchange.com/2.2/questions?order=asc&sort=activity&tagged=python%3Bfunction%3Bdefinition&site=stackoverflow'

r = requests.get(url = URL)
data = r.json()
messages = []
for item in data['items']:
  messages.append(item['title'])

messages.append(input) 

#### Downloading the pre-trained "Universal Sentence Encoder" from tensorflow hub

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 

embed = hub.Module(module_url)

Instructions for updating:
Colocations handled automatically by placer.


W0603 11:16:44.479179  6716 deprecation.py:323] From d:\python\python37\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


#### Converting sentences to embeddings and computing the inner product to calculate similarity

In [7]:
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  message_embeddings = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})
  per = np.inner(message_embeddings, message_embeddings[-1:])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 11:16:46.052101  6716 saver.py:1483] Saver not created because there are no variables in the graph to restore


#### Sorting the results according to similarity

In [44]:
dictItems = []
i = 0
for i in range(0, len(per)-1 ):
    temp = { "probability" : per.item(i), "title" : messages[i] }
    dictItems.append(temp)
    
sortedSearch = sorted(dictItems, key = lambda i: i['probability'], reverse = True)

#### Printing the top 5 results

In [57]:
for i in range(0,5):
    print(f'{i+1}. {sortedSearch[i].get("title")}')

1. How to define function of multiple parameters and tuple in Python?
2. Regarding Function in Python
3. Python definition function parameter passing
4. Function that defines a function in python
5. Multiplying Functions in Python
